In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 400 ~/.kaggle/kaggle.json

In [ ]:
!pip install -U kaggle
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset
!unzip -qq stanford-dogs-dataset.zip -d data/

100% 748M/750M [00:06<00:00, 171MB/s]
100% 750M/750M [00:06<00:00, 113MB/s]


In [ ]:
!mkdir -p checkpoint

In [ ]:
import os, sys
import shutil
images_dir = '/content/data/images/Images'
listdir = os.listdir(images_dir)
di = []
for folder in listdir:
  #print(folder, len(os.listdir(images_dir + '/' + folder)))
  di.append((folder, len(os.listdir(images_dir + '/' + folder)) ))

di.sort(key=lambda x: x[1], reverse=True)
!rm -rf /content/data/images/CloneImages
#print(*di[:5], sep="\n")
clone_dir = '/content/data/images/CloneImages'
os.makedirs('/content/data/images/CloneImages',exist_ok = True)
for folder, _ in di[0:20]:
  shutil.copytree(images_dir + '/' + folder, clone_dir + '/' + folder)

In [ ]:
# data.py
import torch.nn as nn
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split



def getTrainValLoader(opt):
  image_transforms = transforms.Compose([
      transforms.RandomRotation(10),
      transforms.RandomHorizontalFlip(p=0.5),
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  target_transforms = transforms.Lambda(lambda y: torch.zeros(opt['num_classes'],
                                                              dtype=torch.float).scatter_(0,
                                                                                         torch.tensor(y),
                                                                                          value=1))
  dataset = ImageFolder(opt['data_folder'],
                        transform=image_transforms,
                        target_transform=target_transforms
                        )

  train_set, valid_set = random_split(dataset, [1 - opt['validation_split'], opt['validation_split']])

  train_loader = DataLoader(train_set, batch_size=opt['batch_size'], shuffle=True, drop_last=True)
  valid_loader = DataLoader(valid_set, batch_size=opt['val_batch_size'], shuffle=False)
  return train_loader, valid_loader
#getTrainValLoader(opt)

In [ ]:
# model.py
import torch.nn as nn
import torch
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torchvision
import torch.nn.functional as F
import datetime
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR


model_dict = {
    'resnet18': torchvision.models.resnet18,
    'resnet34': torchvision.models.resnet34,
    'resnet50': torchvision.models.resnet50,
    'resnet101': torchvision.models.resnet101,
    'resnet152': torchvision.models.resnet152
}
def create_model(opt):
  model_ft = model_dict[opt['model']](progress=False, weights=opt['pretrained_weights'])
  for param in model_ft.parameters():
    param.requires_grad = False
  num_ftrs = model_ft.fc.in_features


  model_ft.fc = nn.Sequential(
      nn.Linear(num_ftrs, opt['num_hidden']),
      nn.ReLU(),
      nn.Dropout(opt['dropout']),
      nn.Linear(opt['num_hidden'], opt['num_classes']),
      nn.Softmax(dim=1)
  )
  return model_ft

def criterion():
    return F.cross_entropy
def evaluate(model, val_loader, opt):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0
    count = 0
    with torch.no_grad():
        for batch_id , test_data in enumerate(val_loader,0):
            count += 1
            data, label = test_data
            #label = F.one_hot(label, num_classes=opt.num_classes)
            if opt['use_gpu']:
                data = data.to("cuda")
                label = label.to("cuda")
            outputs = model(data)
            _, correct_labels = torch.max(label, 1)
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == correct_labels).sum().item()
            running_loss += criterion()(
                outputs.float(), label.float()).item()
    #        print(f"--> Total {total}\n-->batch_id: {batch_id + 1}")
    acc = round(correct/total * 1.0 , 5)
    running_loss /= count
    return running_loss, acc
def train_one_iter(model, optim, train_load, val_loader, opt, epoch, lr_scheduler):
    losses = 0
    total_correct = 0
    total_sample = 0
    model.train()
    with tqdm(train_load,  unit="batch", position=0, leave=True) as tp:
        tp.set_description(f"Epoch {epoch}/{opt['epoch']}")
        for (batch_x, batch_y) in tp:

            optim.zero_grad()
            #batch_y = torch.nn.functional.one_hot(batch_y, num_classes = opt.num_classes)
            if opt['use_gpu']:
                batch_x = batch_x.to("cuda")
                batch_y = batch_y.to("cuda")
            outputs = model(batch_x)
            _, correct_labels = torch.max(batch_y, 1)
            _, predicted = torch.max(outputs.data, 1)
            total = batch_y.size(0)
            correct = (predicted == correct_labels).sum().item()
            total_correct += correct
            total_sample += total
            train_acc = round(total_correct / total_sample, 5)
            loss = criterion()(outputs.float(), batch_y.float())
            loss_item = loss.item()
            losses += loss_item
            loss.backward()
            optim.step()
            tp.set_postfix(loss=loss_item, train_acc=train_acc)
        val_loss, val_acc = evaluate(model, val_loader, opt)
        lr_scheduler.step()
        losses /= len(tp)
        print(f"Epoch {epoch}/{opt['epoch']} is finished with validation accuracy is {val_acc}")

    return model, optim, losses, total_correct / total_sample, val_loss, val_acc
def load_from_checkpoint(option):
    ckpt = torch.load(option['continual_checkpoint_pth'])
    opt = ckpt['opt']
    model = create_model(opt)
    model = model.to(ckpt['device'])
    model.load_state_dict(ckpt['model_state_dict'])
    optim = None
    if opt['optimizer'] == "sgd":
        optim = torch.optim.SGD(model.parameters())
    elif opt['optimizer'] == "adam":
        optim = torch.optim.Adam(model.parameters())
    elif opt['optimizer'] == "adamw":
        optim = torch.optim.AdamW(model.parameters())
    optim.load_state_dict(ckpt['optimizer_state_dict'])

    return model,\
           optim,\
           ckpt['epoch'],\
           ckpt['losses'],\
           ckpt['val_losses'],\
           ckpt['train_acc'],\
           ckpt['val_acc'],\
           opt

def training_process(opt):
    train_loader, val_loader = getTrainValLoader(opt)

    if opt['resume_from_checkpoint']:
      model, optim, init_epoch, losses,  val_losses, train_accuracies, val_accuracies, opt = load_from_checkpoint(opt)
      #best_model = model
      #best_val_acc = ckpt
      #
    else:

      model = create_model(opt)
      best_model = None
      best_val_acc = 0
      best_epoch = -1
      losses = []
      val_accuracies = []
      val_losses = []
      init_epoch = 1
      train_accuracies = []

      optim = None
      if opt['optimizer'] == "sgd":
          optim = torch.optim.SGD(model.parameters(), lr=opt['learning_rate'], weight_decay=opt['weight_decay'], nesterov=True)
      elif opt['optimizer'] == "adam":
          optim = torch.optim.Adam(model.parameters(), lr=opt['learning_rate'], weight_decay=opt['weight_decay'], eps=1e-7)
      elif opt['optimizer'] == "adamw":
          optim = torch.optim.AdamW(model.parameters(), lr=opt['learning_rate'], weight_decay=opt['weight_decay'], eps=1e-7)
    if opt['use_gpu'] and torch.cuda.is_available():
        model = model.to("cuda")



    now = datetime.datetime.now()
    #tags = {"author": "Dung Vo", "data": opt.data}
    #runName = f"Data: {opt.data}_Optimization: {opt.optimizer}_LR: {opt.learning_rate}_Batch Size: {opt.batch_size}_DropOut: {opt.dropout}_{datetime.datetime.strftime(now, format='%Y%m%d%H%M%S')}"


    lr_scheduler = MultiStepLR(optim, milestones=[5, 10, 15, 20, 25, 30], gamma=0.2, verbose=True)
    #reduceLROnPlateau = ReduceLROnPlateau(optim, patience=2, factor=0.1)
    patience_cnt = 0
    for i in range(1, opt['epoch']+1):


        model, optim, epoch_loss, train_acc, val_loss, val_acc = train_one_iter(model,
                                                                                optim, train_loader,
                                                                                val_loader, opt, epoch = i,
                                                                                lr_scheduler=lr_scheduler
                                                                               )
        losses.append(epoch_loss)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        train_accuracies.append(train_acc)
        if i % opt['checkpoint_save_freq'] == 0 and opt['save_checkpoint']:
            torch.save({
                'opt': opt,
                'epoch': i,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict(),
                'losses': losses,
                'val_losses': val_losses,
                'train_acc': train_accuracies,
                'val_acc': val_accuracies,
                'device': "cuda" if opt['use_gpu'] and torch.cuda.is_available() else "cpu"
            }, opt['checkpoint_pth']  + f"/checkpoint_e{i}.pt")
        if val_acc > best_val_acc:
            best_model = model
            best_val_acc = val_acc
            best_epoch = i
            patience_cnt = 0 # Reset the patience_cnt
        else: # Increase the patience for early_stopping if metric does not improve
            patience_cnt += 1
            if patience_cnt > opt['patience']:
                break



    torch.save({
        'opt': opt,
        'model_state_dict': best_model.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'epoch': best_epoch,
        'losses': losses[:best_epoch],
        'val_losses': val_losses[:best_epoch],
        'train_acc': train_accuracies[:best_epoch],
        'val_acc': val_accuracies[:best_epoch],
        'device': "cuda" if opt['use_gpu'] and torch.cuda.is_available() else "cpu"

    }, opt['checkpoint_pth'] + "/final_model.pt")
    return best_model, losses, val_losses, train_accuracies, val_accuracies

In [ ]:

opt = {
    'data_folder': '/content/data/images/CloneImages',
    'num_classes': 20,
    'batch_size': 32,
    'val_batch_size': 128,
    'checkpoint_pth': './checkpoint/',
    'resume_from_checkpoint': False,
    'model': 'resnet50',
    'pretrained_weights': 'IMAGENET1K_V2',
    'epoch': 400,
    'early_stopping': True,
    'patience': 10,
    'optimizer': 'adam',
    'learning_rate': 0.005,
    'weight_decay': 4e-3,
    'use_gpu': True,
    'checkpoint_save_freq': 5,
    'validation_split': 0.2,
    'dropout': 0.35,
    'num_hidden': 512,
    'save_checkpoint': True,
    'continual_checkpoint_pth': '/content/checkpoint/final_model.pt'
    #'continue_train': False
}



In [ ]:
model, losses, val_losses, train_acc, val_acc = training_process(opt)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


Adjusting learning rate of group 0 to 5.0000e-03.


Epoch 1/400: 100%|██████████| 106/106 [00:43<00:00,  2.43batch/s, loss=2.47, train_acc=0.541]


Adjusting learning rate of group 0 to 5.0000e-03.
Epoch 1/400 is finished with validation accuracy is 0.73939


Epoch 2/400: 100%|██████████| 106/106 [00:36<00:00,  2.94batch/s, loss=2.33, train_acc=0.727]


...

Adjusting learning rate of group 0 to 3.2000e-07.
Epoch 33/400 is finished with validation accuracy is 0.90684


Epoch 34/400: 100%|██████████| 106/106 [00:35<00:00,  3.00batch/s, loss=2.18, train_acc=0.922]


Adjusting learning rate of group 0 to 3.2000e-07.
Epoch 34/400 is finished with validation accuracy is 0.91509


Epoch 35/400: 100%|██████████| 106/106 [00:35<00:00,  3.00batch/s, loss=2.25, train_acc=0.92]


Adjusting learning rate of group 0 to 3.2000e-07.
Epoch 35/400 is finished with validation accuracy is 0.91981


In [ ]:
# prune.py
import torch.nn.utils.prune as prune
import torch
import torch.nn as nn
import os
import sys
def measure_module_sparsity(module, weight=True, bias=False, use_mask=False):

    num_zeros = 0
    num_elements = 0

    if use_mask == True:
        for buffer_name, buffer in module.named_buffers():
            if "weight_mask" in buffer_name and weight == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
            if "bias_mask" in buffer_name and bias == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
    else:
        for param_name, param in module.named_parameters():
            if "weight" in param_name and weight == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()
            if "bias" in param_name and bias == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()

    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity


def measure_global_sparsity(model,
                            weight=True,
                            bias=False,
                            conv2d_use_mask=False,
                            linear_use_mask=False):

    num_zeros = 0
    num_elements = 0

    for module_name, module in model.named_modules():

        if isinstance(module, torch.nn.Conv2d):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=conv2d_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

        elif isinstance(module, torch.nn.Linear):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=linear_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity


def remove_parameters(model, opt):

    for module_name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            try:
                prune.remove(module, "weight")
            except:
                pass
            try:
                prune.remove(module, "bias")
            except:
                pass
        elif isinstance(module, torch.nn.Linear) and module_name != opt['ignore_prune']:
            try:
                prune.remove(module, "weight")
            except:
                pass
            try:
                prune.remove(module, "bias")
            except:
                pass

    return model
def prune_model(model, prune_opt):
    for name, module in model.named_modules():
        # prune 20% of connections in all 2D-conv layers
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=prune_opt['conv_rate'])
            prune.remove(module, name='weight')
        if isinstance(module, torch.nn.Linear) and name != prune_opt['ignore_prune']: # Do not prune last layer
            prune.l1_unstructured(module, name='weight', amount=prune_opt['linear_rate'])
            prune.remove(module, name='weight')
    return model

#evaluate(model.to("cuda"), val_loader, opt)

In [ ]:
prune_opt = {
    'data_folder': '/content/data/images/CloneImages',
    'num_classes': 20,
    'batch_size': 32,
    'val_batch_size': 128,
    'checkpoint_pth': './checkpoint/pruned/',
    'continual_checkpoint_pth': '/content/checkpoint/final_model.pt',

    #'resume_from_checkpoint': False,
    'model': 'resnet50',
    'pretrained_weights': 'IMAGENET1K_V2',
    'epoch': 400,
    'early_stopping': True,
    'patience': 10,
    'optimizer': 'adam',
    'learning_rate': 0.005,
    'weight_decay': 4e-3,
    'use_gpu': True,
    'checkpoint_save_freq': 5,
    'validation_split': 0.2,
    'dropout': 0.35,
    'num_hidden': 512,
    'save_checkpoint': True,
    'num_iterations': 2,
    'conv_rate': 0.4,
    'linear_rate': 0.4,
    'ignore_prune': 'fc.3'
    #'continue_train': False

}
def iterative_pruning_finetuning(opt):
    train_loader, val_loader = getTrainValLoader(opt)
    model, _, _, _, _, _, _, _ = load_from_checkpoint(opt)
    os.makedirs(opt['checkpoint_pth'], exist_ok = True)

    for cnt in range(1, opt['num_iterations'] + 1):
      print(f"==== Start process for pruning at iteration number {cnt} ====\n\n")
      print(f"====                        Pruning                      ====")

      model = prune_model(model, opt) # Passed

      print(f"====                        Pruned                       ====\n\n")
      print(f"====           Evaluate pruned model performance         ====")


      _, _, sparsity = measure_global_sparsity(model) # Passed
      _, val_accuracy = evaluate(model, val_loader, opt)
      print(f"Pruned model's performance: {round((val_accuracy * 100), 2)}% accuracy on validation dataset")
      print(f"Sparsity {sparsity * 100}%")
      print(f"====               Evaluation process ended              ====\n\n")
      print(f"====                       Finetune                      ====")




      optim = None

      if opt['optimizer'] == "sgd":
          optim = torch.optim.SGD(model.parameters(), lr=opt['learning_rate'], weight_decay=opt['weight_decay'], nesterov=True)
      elif opt['optimizer'] == "adam":
          optim = torch.optim.Adam(model.parameters(), lr=opt['learning_rate'], weight_decay=opt['weight_decay'], eps=1e-7)
      elif opt['optimizer'] == "adamw":
          optim = torch.optim.AdamW(model.parameters(), lr=opt['learning_rate'], weight_decay=opt['weight_decay'], eps=1e-7)
      lr_scheduler = MultiStepLR(optim, milestones=[5, 10, 15, 20, 25, 30], gamma=0.2, verbose=True)
      #reduceLROnPlateau = ReduceLROnPlateau(optim, patience=2, factor=0.1)
      patience_cnt = 0

      val_losses, losses, val_accuracies, train_accuracies = [], [], [], []

      best_val_acc = -1
      best_epoch = -1

      for i in range(1, opt['epoch']+1):

          model, optim, epoch_loss, train_acc, val_loss, val_acc = train_one_iter(model,
                                                                                  optim, train_loader,
                                                                                  val_loader, opt, epoch = i,
                                                                                  lr_scheduler=lr_scheduler
                                                                                )
          losses.append(epoch_loss)
          val_losses.append(val_loss)
          val_accuracies.append(val_acc)
          train_accuracies.append(train_acc)

          if i % opt['checkpoint_save_freq'] == 0 and opt['save_checkpoint']:
              torch.save({
                  'opt': opt,
                  'epoch': i,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optim.state_dict(),
                  'losses': losses,
                  'val_losses': val_losses,
                  'train_acc': train_accuracies,
                  'val_acc': val_accuracies,
                  'device': "cuda" if opt['use_gpu'] and torch.cuda.is_available() else "cpu"
              }, opt['checkpoint_pth']  + f"/checkpoint_e{i}.pt")
          if val_acc > best_val_acc:
              best_model = model
              best_val_acc = val_acc
              best_epoch = i
              patience_cnt = 0 # Reset the patience_cnt
          else: # Increase the patience for early_stopping if metric does not improve
              patience_cnt += 1
              if patience_cnt > opt['patience']:
                  break



      torch.save({
          'opt': opt,
          'model_state_dict': best_model.state_dict(),
          'optimizer_state_dict': optim.state_dict(),
          'epoch': best_epoch,
          'losses': losses[:best_epoch],
          'val_losses': val_losses[:best_epoch],
          'train_acc': train_accuracies[:best_epoch],
          'val_acc': val_accuracies[:best_epoch],
          'device': "cuda" if opt['use_gpu'] and torch.cuda.is_available() else "cpu"

      }, opt['checkpoint_pth'] + f"/final_model_prune_iter_{cnt}.pt")

    model = remove_parameters(model, opt)
    torch.save({
      'opt': opt,
      'model_state_dict': best_model.state_dict(),
      'device': "cuda" if opt['use_gpu'] and torch.cuda.is_available() else "cpu"

    }, opt['checkpoint_pth'] + f"/final_model_prune.pt")

In [ ]:
iterative_pruning_finetuning(prune_opt)

==== Start process for pruning at iteration number 1 ====


====                        Pruning                      ====
====                        Pruned                       ====


====           Evaluate pruned model performance         ====
Pruned model's performance: 3.54% accuracy on validation dataset
Sparsity 78.25557173515183%
====               Evaluation process ended              ====


====                       Finetune                      ====
Adjusting learning rate of group 0 to 5.0000e-03.


Epoch 1/400: 100%|██████████| 106/106 [00:37<00:00,  2.85batch/s, loss=2.75, train_acc=0.419]


Adjusting learning rate of group 0 to 5.0000e-03.
Epoch 1/400 is finished with validation accuracy is 0.41274


Epoch 2/400: 100%|██████████| 106/106 [00:37<00:00,  2.85batch/s, loss=2.64, train_acc=0.443]


Adjusting learning rate of group 0 to 5.0000e-03.
Epoch 2/400 is finished with validation accuracy is 0.50825


...

Adjusting learning rate of group 0 to 3.2000e-07.
Epoch 39/400 is finished with validation accuracy is 0.5566


Epoch 40/400: 100%|██████████| 106/106 [00:34<00:00,  3.08batch/s, loss=2.6, train_acc=0.58]


Adjusting learning rate of group 0 to 3.2000e-07.
Epoch 40/400 is finished with validation accuracy is 0.56486


Epoch 41/400: 100%|██████████| 106/106 [00:34<00:00,  3.12batch/s, loss=2.55, train_acc=0.585]


Adjusting learning rate of group 0 to 3.2000e-07.
Epoch 41/400 is finished with validation accuracy is 0.57075
